In [ ]:
SET TARGET_DATABASE   = '&{ dbsname }';
SET DEPLOY_SCHEMA     = '&{ depname }';
SET DEPLOY_ROLE_OWNER = 'APP_&{ depname }_OWNER';
SET APP_ROLE_TYPE_01  = 'APP_&{ depname }_ROL01';
SET APP_ROLE_TYPE_02  = 'APP_&{ depname }_ROL02';
SET APP_NB_CONTROL    = 'APP_&{ depname }_CONTROL';
SET SERVICE_NAME      = 'APP_&{ depname }_SVC';
SET SERVICE_RUN_USER  = 'APP_&{ depname }_USER';
SET SERVICE_RUN_POOL  = 'APP_&{ depname }_POOL';
SET SERVICE_RUN_VWHS  = 'APP_&{ depname }_WH';
SET EXT_ACC_INT_NAME  = 'APP_&{ depname }_EXASINT';
SET EXT_ACC_NET_RULE  = 'APP_&{ depname }_NETRULE';
SET SERVICE_REF       = 'app-&{ depname }';

SELECT * FROM (VALUES 
	 ('TARGET_DATABASE:    [' || $TARGET_DATABASE   ||']')
	,('DEPLOY_SCHEMA:      [' || $DEPLOY_SCHEMA     ||']')
	,('DEPLOY_ROLE_OWNER:  [' || $DEPLOY_ROLE_OWNER ||']')
	,('APP_ROLE_TYPE_01:   [' || $APP_ROLE_TYPE_01  ||']')
	,('APP_ROLE_TYPE_02:   [' || $APP_ROLE_TYPE_02  ||']')
	,('APP_NB_CONTROL:     [' || $APP_NB_CONTROL    ||']')
	,('SERVICE_NAME:       [' || $SERVICE_NAME      ||']')
	,('SERVICE_RUN_USER:   [' || $SERVICE_RUN_USER  ||']')
	,('SERVICE_RUN_POOL:   [' || $SERVICE_RUN_POOL  ||']')
	,('SERVICE_RUN_VWHS:   [' || $SERVICE_RUN_VWHS  ||']')
	,('EXT_ACC_INT_NAME:   [' || $EXT_ACC_INT_NAME  ||']')
	,('EXT_ACC_NET_RULE:   [' || $EXT_ACC_NET_RULE  ||']')
	,('SERVICE_REF:        [' || $SERVICE_REF       ||']')
) AS t (VARS)

In [ ]:
/*
USE ROLE ACCOUNTADMIN;
USE DATABASE IDENTIFIER($TARGET_DATABASE);

CREATE SCHEMA IF NOT EXISTS IDENTIFIER($DEPLOY_SCHEMA);
USE SCHEMA IDENTIFIER($DEPLOY_SCHEMA);

DROP USER IF EXISTS IDENTIFIER($SERVICE_RUN_USER);
DROP ROLE IF EXISTS IDENTIFIER($DEPLOY_ROLE_OWNER);
DROP ROLE IF EXISTS IDENTIFIER($APP_ROLE_TYPE_01);

DROP NOTEBOOK IF EXISTS IDENTIFIER($APP_NB_CONTROL);

ALTER SERVICE IF EXISTS IDENTIFIER($SERVICE_NAME) SUSPEND;
DROP SERVICE IF EXISTS IDENTIFIER($SERVICE_NAME) FORCE;

ALTER COMPUTE POOL IF EXISTS IDENTIFIER($SERVICE_RUN_POOL) SUSPEND;
DROP COMPUTE POOL IF EXISTS IDENTIFIER($SERVICE_RUN_POOL);

DROP WAREHOUSE IF EXISTS IDENTIFIER($SERVICE_RUN_VWHS);

DROP EXTERNAL ACCESS INTEGRATION IF EXISTS IDENTIFIER($EXT_ACC_INT_NAME);
DROP NETWORK RULE IF EXISTS IDENTIFIER($EXT_ACC_NET_RULE);

DROP SCHEMA IF EXISTS IDENTIFIER($DEPLOY_SCHEMA);
*/

In [ ]:
USE ROLE IDENTIFIER($DEPLOY_ROLE_OWNER);

USE DATABASE IDENTIFIER($TARGET_DATABASE);
USE SCHEMA IDENTIFIER($DEPLOY_SCHEMA);
USE WAREHOUSE IDENTIFIER($SERVICE_RUN_VWHS);

ALTER SERVICE IF EXISTS IDENTIFIER($SERVICE_NAME) SUSPEND;
ALTER COMPUTE POOL IF EXISTS IDENTIFIER($SERVICE_RUN_POOL) SUSPEND;
ALTER WAREHOUSE IF EXISTS IDENTIFIER($SERVICE_RUN_VWHS) SUSPEND;

In [ ]:
USE ROLE IDENTIFIER($DEPLOY_ROLE_OWNER);

USE DATABASE IDENTIFIER($TARGET_DATABASE);
USE SCHEMA IDENTIFIER($DEPLOY_SCHEMA);
USE WAREHOUSE IDENTIFIER($SERVICE_RUN_VWHS);

ALTER WAREHOUSE IF EXISTS IDENTIFIER($SERVICE_RUN_VWHS) RESUME IF SUSPENDED;
ALTER COMPUTE POOL IF EXISTS IDENTIFIER($SERVICE_RUN_POOL) RESUME;
ALTER SERVICE IF EXISTS IDENTIFIER($SERVICE_NAME) RESUME;

In [ ]:
USE ROLE IDENTIFIER($DEPLOY_ROLE_OWNER);

USE DATABASE IDENTIFIER($TARGET_DATABASE);
USE SCHEMA IDENTIFIER($DEPLOY_SCHEMA);
USE WAREHOUSE IDENTIFIER($SERVICE_RUN_VWHS);
-- show service deployment specs
SELECT $1 from @SPECS/service.yaml; 
-- show all services
SHOW SERVICES;
-- check the status of service
CALL SYSTEM$GET_SERVICE_STATUS($SERVICE_NAME);
-- check the logs in the docker containers
CALL SYSTEM$GET_SERVICE_LOGS($SERVICE_NAME, '0', $SERVICE_REF, 1000);
-- check service containers
SHOW SERVICE CONTAINERS IN SERVICE IDENTIFIER($SERVICE_NAME);
-- check service endpoints
SHOW ENDPOINTS IN SERVICE IDENTIFIER($SERVICE_NAME);